# In class exercise
[join and compute avg age per gender](#join)


In [1]:
#from pyspark.context import SparkContext
#from pyspark.sql.session import SparkSession
#sc = SparkContext('local')
#spark = SparkSession(sc)

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("joins").getOrCreate()

In [ ]:
file_loc = 'gs://wac-buck/notebooks/jupyter/pyspark/ages.csv'
age = spark.read.csv(file_loc, inferSchema=True, header=True)

In [ ]:
file_loc = 'gs://wac-buck/notebooks/jupyter/pyspark/genders.csv'
gender = spark.read.csv(file_loc, inferSchema=True, header=True)

In [4]:
age.show()

+---+---+
| ID|Age|
+---+---+
|  1| 42|
|  2|  2|
|  3| 34|
|  4| 12|
|  5| 34|
|  6| 67|
|  7| 34|
|  8| 78|
|  9| 56|
| 10| 12|
| 11| 45|
| 12|  2|
| 13| 89|
| 14| 23|
| 15| 45|
| 16| 32|
| 17| 12|
| 18| 35|
| 19| 67|
| 20| -1|
+---+---+



In [5]:
gender.show()

+---+------+
| ID|Gender|
+---+------+
|  1|     M|
|  2|     M|
|  3|     M|
|  4|     F|
|  5|     F|
|  6|     M|
|  7|     M|
|  8|     F|
|  9|     M|
| 10|     F|
| 11|     F|
| 12|     F|
| 13|     M|
| 14|     M|
| 15|     F|
| 16|     M|
| 17|     F|
| 18|     X|
| 19|     M|
| 20|     F|
+---+------+



In [7]:
age.columns

['ID', 'Age']

In [8]:
age.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Age: integer (nullable = true)



In [10]:
age.count()

20

In [11]:
age.select('ID').show(5)

+---+
| ID|
+---+
|  1|
|  2|
|  3|
|  4|
|  5|
+---+
only showing top 5 rows



In [12]:
age.filter(age['Age']>30).show()

+---+---+
| ID|Age|
+---+---+
|  1| 42|
|  3| 34|
|  5| 34|
|  6| 67|
|  7| 34|
|  8| 78|
|  9| 56|
| 11| 45|
| 13| 89|
| 15| 45|
| 16| 32|
| 18| 35|
| 19| 67|
+---+---+



In [14]:
age.filter((age['Age'] > 30) & (age['ID'] < 11)).show()

+---+---+
| ID|Age|
+---+---+
|  1| 42|
|  3| 34|
|  5| 34|
|  6| 67|
|  7| 34|
|  8| 78|
|  9| 56|
+---+---+



In [16]:
gender.groupBy('Gender').max().show()

+------+-------+
|Gender|max(ID)|
+------+-------+
|     F|     20|
|     M|     19|
|     X|     18|
+------+-------+



<a id='join'></a>
## join the two dataframes

In [19]:
df = age.join(gender, 'ID')
df.show()

+---+---+------+
| ID|Age|Gender|
+---+---+------+
|  1| 42|     M|
|  2|  2|     M|
|  3| 34|     M|
|  4| 12|     F|
|  5| 34|     F|
|  6| 67|     M|
|  7| 34|     M|
|  8| 78|     F|
|  9| 56|     M|
| 10| 12|     F|
| 11| 45|     F|
| 12|  2|     F|
| 13| 89|     M|
| 14| 23|     M|
| 15| 45|     F|
| 16| 32|     M|
| 17| 12|     F|
| 18| 35|     X|
| 19| 67|     M|
| 20| -1|     F|
+---+---+------+



## compute average age per gender

### using just groupby

In [20]:
df.groupBy('Gender').mean().show()

+------+------------------+------------------+
|Gender|           avg(ID)|          avg(Age)|
+------+------------------+------------------+
|     F|11.333333333333334|26.555555555555557|
|     M|               9.0|              44.6|
|     X|              18.0|              35.0|
+------+------------------+------------------+



In [23]:
df.filter((df['Gender'] == 'M') | (df['Gender'] == 'F')).select(['Age', 'Gender']).groupBy('Gender').mean().show()

+------+------------------+
|Gender|          avg(Age)|
+------+------------------+
|     F|26.555555555555557|
|     M|              44.6|
+------+------------------+



### using groupby and agg

In [30]:
df.groupBy('Gender').agg({'Age':'mean'}).show()

+------+------------------+
|Gender|          avg(Age)|
+------+------------------+
|     F|26.555555555555557|
|     M|              44.6|
|     X|              35.0|
+------+------------------+



In [31]:
df.filter((df['Gender'] == 'M') | (df['Gender'] == 'F')).groupBy('Gender').agg({'Age':'mean'}).show()

+------+------------------+
|Gender|          avg(Age)|
+------+------------------+
|     F|26.555555555555557|
|     M|              44.6|
+------+------------------+



### using sql notation

In [39]:
df.createOrReplaceTempView('people')

In [40]:
spark.sql(
'''
SELECT *
FROM people
'''
).show()

+---+---+------+
| ID|Age|Gender|
+---+---+------+
|  1| 42|     M|
|  2|  2|     M|
|  3| 34|     M|
|  4| 12|     F|
|  5| 34|     F|
|  6| 67|     M|
|  7| 34|     M|
|  8| 78|     F|
|  9| 56|     M|
| 10| 12|     F|
| 11| 45|     F|
| 12|  2|     F|
| 13| 89|     M|
| 14| 23|     M|
| 15| 45|     F|
| 16| 32|     M|
| 17| 12|     F|
| 18| 35|     X|
| 19| 67|     M|
| 20| -1|     F|
+---+---+------+



In [55]:
spark.sql(
'''
SELECT Gender, avg(Age)
FROM people
WHERE Gender IN ('M','F')
GROUP BY Gender
'''
).show()

+------+------------------+
|Gender|          avg(Age)|
+------+------------------+
|     F|26.555555555555557|
|     M|              44.6|
+------+------------------+



In [58]:
age.filter('age>30').join(gender,'ID').show()

+---+---+------+
| ID|Age|Gender|
+---+---+------+
|  1| 42|     M|
|  3| 34|     M|
|  5| 34|     F|
|  6| 67|     M|
|  7| 34|     M|
|  8| 78|     F|
|  9| 56|     M|
| 11| 45|     F|
| 13| 89|     M|
| 15| 45|     F|
| 16| 32|     M|
| 18| 35|     X|
| 19| 67|     M|
+---+---+------+

